In [4]:
%load_ext autoreload
%autoreload 2

import matplotlib
#matplotlib.use('Agg')
%matplotlib tk
%autosave 180

#
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import matplotlib.cm as cm
from matplotlib import gridspec
from scipy import signal

import numpy as np
import pandas as pd
import os
import shutil
import cv2
from tqdm import trange

#import glob2

from scipy.io import loadmat
import scipy
import h5py
#import hdf5storage
import csv
import sleap

from visualize import Visualize
from track import Track


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Autosaving every 180 seconds


In [2]:
###################################################
###################################################
###################################################
# 
def save_two_animal_slp(animal_ids, fname_slp):
    # select first to animals
    labels = sleap.load_file(fname_slp)

    # Change this to specify which pair of tracks to save:
    keep_tracks = [labels.tracks[animal_ids[0]], labels.tracks[animal_ids[1]]]

    # Remove instances that aren't in keep_tracks
    for lf in labels:
        lf.instances = [inst for inst in lf.instances if inst.track in keep_tracks]

    # Keep only those tracks in the labels
    labels.tracks = keep_tracks

    # Remove frames that are now empty due to the track filtering
    labels.remove_empty_frames()

    # Save the result with the specified tracks in the filename
    labels.save(fname_slp[:-4] + "_"+str(animal_ids[0])+"_"+str(animal_ids[1])+".slp")
    
#
fname_slp = '/media/cat/256GB/dan/simba/2020_07_31_20_15_17_668937_compressed_defished_shrink_cropped.slp'
fname_vid = '/media/cat/256GB/dan/simba/2020_07_31_20_15_17_668937_compressed_defished_shrink_cropped.mp4'

#
animal_ids = [0,1]
save_two_animal_slp(animal_ids, fname_slp)

#
print ("DONE...")

In [3]:
######################################################## 
############## GENERATE FIXED TRACK MOVIE ##############
########################################################
#
#fps = 25
shrink = 1 # shrink image factor
fname_video_out = 'fixed'
fname_video = '/media/cat/256GB/dan/simba/2020_07_31_20_15_17_668937_compressed_defished_shrink_cropped.mp4'

# 
fname_slp = '/media/cat/256GB/dan/simba/2020_07_31_20_15_17_668937_compressed_Night.slp'
#fname_slp = '/media/cat/256GB/dan/simba/2020_07_31_20_15_17_668937_compressed_defished_shrink_cropped_0_1.slp'

#
track = Track(fname_slp)
track.animal_ids = [0,1,2,3,4,5]
track.tracks_names = ['female','male','pup1','pup2','pup3','pup4']
track.recompute_spine_centres=True
track.verbose = True                         # gives additional printouts
track.n_animals = len(track.animal_ids)      # number of animals
track.filter_width = 10                      # this is the median filter width in frames; e.g. 10 ~=0.4 seconds
                                             # higher values provide more stability, but less temporally precise locations
# 
track.load_tracks()

####################################################
### OPTIONAL - MEDIAN FILTER ALL TRACKS ############
####################################################
if True:
    track.filter_tracks()
    #track.filter_tracks_spines()
            
####################################################
### OPTIONAL - ALGORITHM TO REASSIGN CHUNKING ######
####################################################
if True:

    # makes scores based on .slp output? (to check)
    track.get_scores()

    # uses track_spines to break up all the data into continuous chunks
    track.max_jump_single_frame = 30  # max distance in pixels (?) that an animal can move in a single frame
    track.make_tracks_chunks()        

    # deletig very short chunks of track that are orphaned..
    min_chunk_len = 8
    track.del_short_chunks(min_chunk_len)

    ############## FIX TRACKS PARAMS #############
    track.time_threshold = 25       # window to search for nearest chunks, about 1sec seems fair...
    track.safe_chunk_length = 15    # chunks this long will not change id
    track.min_chunk_len = 4         # min length of chukn to be used for anchoring/correcting
    track.max_distance_merge = 75   # max pix diff allowed for merging when using model; not just for neighbouring frames
    # track.memory_length = 25      # how many frames back is it ok to remember a prev animal
    track.verbose = False
    track.update_tracks = True
    
    # parameters for fixing track chunking
    track.max_time_automerge = 3      # time to automerget chunks from same animal ???
    track.max_dist_automerge = 25     # distance to auto merge chunks from same animal separated by single time skip


    track.fix_tracks()

else:
    # recompute spine centres from scratch
    track.get_track_spine_centers()
    
####################################################
### OPTIONAL - MEDIAN FILTER SPINE CENTRES #########
####################################################
#if True:
#    track.filter_tracks_spines() 
    
####################################################
########## MAKE MOVIES OF SPINE CENTRES ############
####################################################
vis = Visualize()
start = 0   # frame to start
end = 5000   # frame to end
fps = 25

vis.make_video_centroid(track.tracks_spine,                # visualize centres locations
                        fname_video,
                        fname_video_out,
                        start,
                        end,
                        fps,
                        shrink) 

# 
print ("DONE...")

... Making tracks chunks...
Deleting chunks <  8
... Fixing tracks...


  0%|          | 9/5000 [00:00<01:01, 80.91it/s]

UPDATING TRACKS
width, heigh:  900.0 700.0
Histories:  (6, 5, 2)


100%|██████████| 5000/5000 [01:00<00:00, 83.25it/s]

DONE...


In [28]:
#################################################################
###### MAKE HUNGARIAN ALGORITHM ASSEMBLED ANIMAL VIDEOS #########
#################################################################

#
fname_traces_inferences = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/2020-3-16_12_54_07_193951_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full_traces_inferences_0_89988.npz'
reassembled = np.load(fname_traces_inferences)
#reassembled = np.load('/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_57_12_418305_compressed/pickle/2020-3-16_12_57_12_418305_compressedDLC_resnet50_madeline_july2Jul2shuffle1_100000_full_traces_inferences.npz')
tracesx_re = reassembled['tracesx'].T
tracesy_re = reassembled['tracesy'].T
print ("Lenght of tracessx: ", tracesx_re.shape)

# OPTIONAL MAKE VIDEO TO REVIEW ASSEMBLED VS. INFERENCE LABELS (PRE-FIX)
# colors have weird inversion; red is blue and cyan is yellow
#colors_4 = ['blue','red','cyan','green','pink','orange']

#          pup1     pup2    female  male
colors_4= ['orange','green', 'blue', 'red', 'cyan']
colors_5= ['red','blue', 'cyan', 'green', 'yellow']

video_name = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/2020-3-16_12_54_07_193951_compressed.avi'
#video_name = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_57_12_418305_compressed/2020-3-16_12_57_12_418305_compressed.avi'
original_vid = cv2.VideoCapture(video_name)

# video sizes
size_vid = np.array([1280,1024])
scale = 1
dot_size = 8//scale
dot_size2 = 4//scale

#

#dot_size = 10//scale
# setup cutoff 
pcutoff = 0.01

# go through first videos
from tqdm import trange

start = 1
end = tracesx_re.shape[0]
#end = 1001
n_networks = 5
comments=False

out_dir = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/2020-3-9_08_18_49_128168/'
fname_out = video_name[:-4]+"_CNN_"+str(start)+"_"+str(end)+".mp4"
fourcc = cv2.VideoWriter_fourcc('M','P','E','G')
video_out = cv2.VideoWriter(fname_out,fourcc, 25, (size_vid[0]//scale,size_vid[1]//scale), True)
original_vid.set(cv2.CAP_PROP_POS_FRAMES, start)

# LOOP 
font = cv2.FONT_HERSHEY_PLAIN
for n in trange(start,end, 1):
    ret, frame = original_vid.read()
    #print (n, frame.shape)
    cv2.putText(frame, str(n), (50, 100), font, 5, (255, 255, 0), 5)
    frame = frame[::scale, ::scale]
        
    animal_ids, animal_ids2, dirs = get_animal_ids(n,
                                chain_ids,
                                vals,
                                classes,
                                comments)
    shift=20
    #titles = np.zeros(n_networks)
    for k in range(0, 14*n_networks,14):
    #for k in range(14*n_networks):
        y = tracesx_re[n,k:k+14]
        x = tracesy_re[n,k:k+14]
        
        #if animal_ids[k//14]==None:
        #    continue
            
        idx = np.where(np.isnan(x))[0]
        x = np.delete(x,idx,0)
        y = np.delete(y,idx,0)
        idx = np.where(x==0)[0]
        x = np.delete(x,idx,0)
        y = np.delete(y,idx,0)
        
        #if np.isnan(x) or np.isnan(y):
        #    continue
        
        if x.shape[0]==0:
            continue
        #print (x)
        x1=np.int32(x)//scale
        y1=np.int32(y)//scale
        
        flag=True
        for x, y in zip(x1,y1):
            if animal_ids[k//14]==1E10:
                continue
                
            if animal_ids[k//14]!= None:
                frame[x-dot_size:x+dot_size,y-dot_size:y+dot_size]= (np.float32(
                    #matplotlib.colors.to_rgb(colors_4[k//14]))*255.).astype('uint8')
                    #matplotlib.colors.to_rgb('white'))*255.).astype('uint8')
                    matplotlib.colors.to_rgb(colors_5[animal_ids[k//14]]))*255.).astype('uint8')

            if animal_ids2[k//14]!= None:
                frame[x-dot_size2+shift:x+shift+dot_size2,y-dot_size2:y+dot_size2]= (np.float32(
                    #matplotlib.colors.to_rgb(colors_4[k//14]))*255.).astype('uint8')
                    #matplotlib.colors.to_rgb('white'))*255.).astype('uint8')
                    matplotlib.colors.to_rgb(colors_5[animal_ids2[k//14]]))*255.).astype('uint8')
        
        #print (colors_4[k])
        #frame[y-dot_size:y+dot_size,x-dot_size:x+dot_size]= (np.float32(
        #    matplotlib.colors.to_rgb(colors_4[z//14]))*255.).astype('uint8')

        #if titles[k//14]==0:
            if flag:
                cv2.putText(frame, str(dirs[k//14]), (y, x), font, 2, (255, 255, 0), 5)
                flag=False
    #print ("")
    video_out.write(frame)

    #print ("")

video_out.release()
original_vid.release()
#cv2.destroyAllWindows()


  0%|          | 0/89987 [00:00<?, ?it/s]

# of lableed frames:  (292353,)
Lenght of tracessx:  (89988, 70)


100%|██████████| 89987/89987 [49:17<00:00, 30.43it/s]  
